In [ ]:
import pandas as pd
df = pd.read_excel("Itens_Agrupados_com_Dia_Semana.xlsx")
df.head()


In [ ]:
import pandas as pd  
import locale  

# Definir o locale para português
locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")  

# Carregar o arquivo Excel
df = pd.read_excel("Itens_Agrupados_com_Dia_Semana.xlsx")  

# Converter a coluna "Data" para o formato de data
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")  

# Criar a coluna "Dia da Semana" em português
df["Dia da Semana"] = df["Data"].dt.strftime("%A")

# Exibir as primeiras linhas para verificar
print(df.head())


In [ ]:
# Verificar se há valores nulos
print(df.isnull().sum())

# Remover linhas com valores nulos, se necessário
df = df.dropna()

# Converter colunas numéricas corretamente
df["Valor Unitário"] = df["Valor Unitário"].astype(float)
df["Qtde"] = df["Qtde"].astype(int)
df["Total"] = df["Total"].astype(float)

# Verificar os tipos de dados após a conversão
print(df.dtypes)

# Exibir estatísticas básicas
print(df.describe())

# Exibir contagem de vendas por dia da semana
print(df["Dia da Semana"].value_counts())

# Visualizar os primeiros dados processados
print(df.head())


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Selecionar apenas as colunas relevantes para a previsão
df_model = df[["Data", "Dia da Semana", "Total"]].copy()

# Normalizar os valores da coluna "Total"
scaler = MinMaxScaler()
df_model["Total"] = scaler.fit_transform(df_model[["Total"]])

# Converter datas para formato numérico (dias desde o início do dataset)
df_model["Data"] = (df_model["Data"] - df_model["Data"].min()).dt.days

# Criar dicionário para mapear dias da semana para números
dias_semana = {
    "segunda-feira": 0, "terça-feira": 1, "quarta-feira": 2, "quinta-feira": 3,
    "sexta-feira": 4, "sábado": 5, "domingo": 6
}
df_model["Dia da Semana"] = df_model["Dia da Semana"].map(dias_semana)

# Criar sequências de entrada para a LSTM
def criar_sequencias(dados, n_passos):
    X, y = [], []
    for i in range(len(dados) - n_passos):
        X.append(dados[i:i + n_passos])
        y.append(dados[i + n_passos])
    return np.array(X), np.array(y)

# Definir quantos dias usar como histórico
n_passos = 30  # Número de dias usados para prever o próximo

# Criar arrays de entrada e saída
X, y = criar_sequencias(df_model[["Data", "Dia da Semana", "Total"]].values, n_passos)

# Dividir em treino (80%) e teste (20%)
tamanho_treino = int(0.95 * len(X))
X_treino, X_teste = X[:tamanho_treino], X[tamanho_treino:]
y_treino, y_teste = y[:tamanho_treino], y[tamanho_treino:]

# Exibir os formatos das matrizes
print("Formato do X_treino:", X_treino.shape)
print("Formato do X_teste:", X_teste.shape)
print("Formato do y_treino:", y_treino.shape)
print("Formato do y_teste:", y_teste.shape)


Split 80/20

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalizar os dados de vendas (assumindo que a coluna normalizada seja 'Total')
scaler = MinMaxScaler()
dados_normalizados = scaler.fit_transform(df[['Total']].values)  # Normaliza apenas a coluna 'Total'

# Definir a janela de tempo (número de dias passados usados para prever o próximo dia)
janela_tempo = 10  # Ajustável conforme necessário

X, y = [], []
for i in range(len(dados_normalizados) - janela_tempo):
    X.append(dados_normalizados[i:i + janela_tempo])
    y.append(dados_normalizados[i + janela_tempo])

X, y = np.array(X), np.array(y)

# Dividir em treino (80%) e teste (20%)
tamanho_treino = int(len(X) * 0.8)
X_treino, X_teste = X[:tamanho_treino], X[tamanho_treino:]
y_treino, y_teste = y[:tamanho_treino], y[tamanho_treino:]

# Exibir as dimensões dos conjuntos
print(f"Formato dos dados de treino: {X_treino.shape}, {y_treino.shape}")
print(f"Formato dos dados de teste: {X_teste.shape}, {y_teste.shape}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Criar o modelo LSTM
modelo = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_treino.shape[1], X_treino.shape[2])),
    LSTM(50, activation='relu', return_sequences=False),
    Dense(25),  # Camada densa intermediária
    Dense(1)  # Saída com uma previsão
])

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo
historico = modelo.fit(X_treino, y_treino, epochs=50, batch_size=16, validation_data=(X_teste, y_teste))

# Exibir resumo do modelo
modelo.summary()


In [ ]:
print(f"Forma de X: {np.array(X).shape}")
print(f"Forma de y: {np.array(y).shape}")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"Tamanho do treino: {len(X_train)}, Tamanho do teste: {len(X_test)}")


In [ ]:
# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo
historico = modelo.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))


In [ ]:
import matplotlib.pyplot as plt

# Plotando o histórico de perda
plt.figure(figsize=(10, 5))
plt.plot(historico.history['loss'], label='Loss (Treinamento)')
plt.plot(historico.history['val_loss'], label='Loss (Validação)')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.title('Evolução da Perda Durante o Treinamento')
plt.grid()
plt.show()


In [ ]:
# Fazer previsões
y_pred = modelo.predict(X_test)


In [ ]:
# Reverter a normalização dos valores previstos e reais
y_pred_original = scaler.inverse_transform(y_pred)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
import matplotlib.pyplot as plt

# Plotando os valores reais vs. previstos
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label="Valores Reais", color="blue")
plt.plot(y_pred_original, label="Previsões", color="red", linestyle="dashed")
plt.xlabel("Amostras")
plt.ylabel("Valor de Venda")
plt.title("Comparação entre Valores Reais e Previstos")
plt.legend()
plt.show()


In [ ]:
print(df.columns)


In [ ]:
df.columns = df.columns.str.strip()
print(df.columns)


In [ ]:
import pandas as pd

# Carregar a planilha
caminho_arquivo = "Itens_Agrupados_com_Dia_Semana.xlsx"
df = pd.read_excel(caminho_arquivo)

# Converter a coluna 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')

# Converter 'Descrição' para string
df['Descrição'] = df['Descrição'].astype(str)

# Converter 'Qtde' para numérico
df['Qtde'] = pd.to_numeric(df['Qtde'], errors='coerce')

# Agrupar por Data e Descrição do Item e somar a Qtde
df_agrupado = df.groupby(['Data', 'Descrição'])['Qtde'].sum().reset_index()

# Ordenar por data
df_agrupado = df_agrupado.sort_values(by='Data')

# Conferir
print(df_agrupado.head())


In [ ]:
import pandas as pd

# Carregar a planilha
caminho_arquivo = "Itens_Agrupados_com_Dia_Semana.xlsx"
df = pd.read_excel(caminho_arquivo)

# Converter a coluna 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')

# Converter 'Descrição' para string
df['Descrição'] = df['Descrição'].astype(str)

# Converter 'Qtde' para numérico
df['Qtde'] = pd.to_numeric(df['Qtde'], errors='coerce')

# Agrupar por Data e Descrição e somar as quantidades
df_agrupado = df.groupby(['Data', 'Descrição'])['Qtde'].sum().reset_index()

# Ordenar por data
df_agrupado = df_agrupado.sort_values(by='Data')

# Transformar em formato de série temporal (linha: data, colunas: itens, valores: qtdes)
df_pivot = df_agrupado.pivot_table(index='Data', columns='Descrição', values='Qtde', fill_value=0)

# Conferir
print(df_pivot.head())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Carregar a planilha
caminho_arquivo = "Itens_Agrupados_com_Dia_Semana.xlsx"
df = pd.read_excel(caminho_arquivo)

# Converter 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')

# Converter 'Descrição' para string
df['Descrição'] = df['Descrição'].astype(str)

# Converter 'Qtde' para numérico
df['Qtde'] = pd.to_numeric(df['Qtde'], errors='coerce')

# Agrupar por Data e Descrição, somando a quantidade
df_agrupado = df.groupby(['Data', 'Descrição'])['Qtde'].sum().reset_index()

# Ordenar por data
df_agrupado = df_agrupado.sort_values(by='Data')

# Pivotar: linha = data, colunas = itens, valores = quantidade
df_pivot = df_agrupado.pivot_table(index='Data', columns='Descrição', values='Qtde', fill_value=0)

# Normalizar os dados
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_pivot)

# Criar sequências de entrada e saída
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Definir janela de tempo (7 dias)
seq_length = 7
X, y = create_sequences(df_scaled, seq_length)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Mostrar as formas dos dados
print("Formato de X_train:", X_train.shape)
print("Formato de X_test:", X_test.shape)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Carregar a planilha
caminho_arquivo = "Itens_Agrupados_com_Dia_Semana.xlsx"
df = pd.read_excel(caminho_arquivo)

# Corrigir formatação da data
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')

# Garantir que a coluna 'Descrição' seja string
df['Descrição'] = df['Descrição'].astype(str)

# Converter 'Qtde' para numérico
df['Qtde'] = pd.to_numeric(df['Qtde'], errors='coerce')

# Agrupar por Data e Descrição (item)
df_agrupado = df.groupby(['Data', 'Descrição'])['Qtde'].sum().reset_index()

# Pivotar: cada linha é um dia, cada coluna é um item
df_pivot = df_agrupado.pivot_table(index='Data', columns='Descrição', values='Qtde', fill_value=0)

# Normalizar os dados
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_pivot)

# Criar sequências de entrada e saída
sequence_length = 7
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled, sequence_length)

# Garantir formato adequado (n_amostras, dias, n_itens)
print("X:", X.shape)  # deve ser (amostras, 7, n_itens)
print("y:", y.shape)  # deve ser (amostras, n_itens)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criar modelo LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(y.shape[1])  # Saída com número de itens
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse')

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Mostrar o resumo
model.summary()


In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
print(type(X_train), type(y_train))
print(type(X_test), type(y_test))


In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fazer previsões com o modelo correto
y_pred = model.predict(X_test)

# Visualizar um único item (ex: o primeiro item da matriz)
item_index = 0  # você pode trocar para 1, 2, 3, etc. para ver outro item
item_name = df_pivot.columns[item_index]  # nome real do item

# Plotar o gráfico para esse item específico
plt.figure(figsize=(10, 5))
plt.plot(y_test[:, item_index], label="Valor Real", color='blue', alpha=0.6)
plt.plot(y_pred[:, item_index], label="Previsão", color='red', alpha=0.6)
plt.xlabel("Amostras")
plt.ylabel("Qtde Vendida (normalizada)")
plt.title(f"Previsão vs Real para o Item: {item_name}")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print("Valores previstos (primeiras 10 amostras):\n", y_pred[:10])
print("Valores reais (primeiras 10 amostras):\n", y_test[:10])


In [ ]:
from sklearn.metrics import r2_score
import numpy as np

# Exibir algumas previsões e valores reais do primeiro item (exemplo)
print("Valores previstos (primeiras 10 amostras - item 0):\n", y_pred[:10, 0])
print("Valores reais (primeiras 10 amostras - item 0):\n", y_test[:10, 0])

# Reverter a escala (caso tenha usado MinMaxScaler)
if 'scaler' in globals():
    y_pred_original = scaler.inverse_transform(y_pred)
    y_test_original = scaler.inverse_transform(y_test)
else:
    y_pred_original = y_pred
    y_test_original = y_test

# Calcular R² para cada item
r2_scores = []
for i in range(y_test.shape[1]):
    r2 = r2_score(y_test_original[:, i], y_pred_original[:, i])
    r2_scores.append(r2)

# Exibir média e melhores itens
print(f"\nMédia dos R² para todos os itens: {np.mean(r2_scores):.4f}")

# Exibir os R² de cada item individualmente
for i, score in enumerate(r2_scores):
    item_name = df_pivot.columns[i]
    print(f"R² para '{item_name}': {score:.4f}")


In [ ]:
y_pred_original = scaler.inverse_transform(y_pred)
y_test_original = scaler.inverse_transform(y_test)


In [ ]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Reduza para 0.0001 se necessário


In [ ]:
print(np.unique(y_pred))  # Se só houver 0, o modelo está preso


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. Aplicar limiar para transformar em classificação binária
limiar = 0.5  # Você pode testar 0.3, 1.0 ou outro valor dependendo do item
y_pred_class = (y_pred_original > limiar).astype(int)
y_test_class = (y_test_original > limiar).astype(int)

# 2. Calcular métricas de classificação
acc = accuracy_score(y_test_class, y_pred_class)
prec = precision_score(y_test_class, y_pred_class, average='macro', zero_division=0)
rec = recall_score(y_test_class, y_pred_class, average='macro', zero_division=0)
f1 = f1_score(y_test_class, y_pred_class, average='macro', zero_division=0)

# 3. Exibir resultados
print("🔍 Avaliação do LSTM como Classificador (com limiar de {:.1f})".format(limiar))
print(f"Acurácia:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")


In [ ]:

lstm_results = {
    "Modelo": "LSTM",
    "Tipo": "Classificação",
    "Acurácia": acc,
    "Precision": prec,
    "Recall": rec,
    "F1 Score": f1
}


In [ ]:
import json

lstm_results = {
    "Modelo": "LSTM",
    "Tipo": "Classificação",
    "Acurácia": acc,
    "Precision": prec,
    "Recall": rec,
    "F1 Score": f1
}

# Salvar em arquivo JSON
with open("resultado_lstm.json", "w") as f:
    json.dump(lstm_results, f)

print("Resultado do LSTM salvo em 'resultado_lstm.json'")
